In [1]:
import spacepy
import spacepy.pybats
import spacepy.pybats.bats

from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
init_notebook_mode(connected = True)
import plotly

import numpy as np
from scipy.interpolate import griddata

This unreleased version of SpacePy is not supported by the SpacePy team.
Qin-Denton/OMNI2 data not found in current format. This module has limited functionality.
Run spacepy.toolbox.update(QDomni=True) to download data


In [2]:
mhd = spacepy.pybats.IdlFile('../../../data/3d__ful_1_t00955900_n4632237.out')
print(mhd)

<class 'spacepy.pybats.IdlFile'>
+
:|____NX (numpy.float64 ())
:|____NY (numpy.float64 ())
:|____NZ (numpy.float64 ())
:|____R (numpy.float64 ())
:|____c (numpy.float64 ())
:|____endian (str [6])
:|____file (str [46])
:|____format (NoneType)
:|____g (numpy.float64 ())
:|____header (str [500])
:|____iter (numpy.int32 ())
:|____ndim (numpy.int32 ())
:|____nparam (numpy.int32 ())
:|____nvar (numpy.int32 ())
:|____p1 (numpy.float64 ())
:|____p2 (numpy.float64 ())
:|____p3 (numpy.float64 ())
:|____runtime (numpy.float32 ())
:|____strtime (str [15])
:|____th (numpy.float64 ())
|____b1x (spacepy.datamodel.dmarray (755136,))
    :|____units (str [2])
|____b1y (spacepy.datamodel.dmarray (755136,))
    :|____units (str [2])
|____b1z (spacepy.datamodel.dmarray (755136,))
    :|____units (str [2])
|____bx (spacepy.datamodel.dmarray (755136,))
    :|____units (str [2])
|____by (spacepy.datamodel.dmarray (755136,))
    :|____units (str [2])
|____bz (spacepy.datamodel.dmarray (755136,))
    :|____uni

In [3]:
mhd.meta

{'file': '../../../data/3d__ful_1_t00955900_n4632237.out',
 'format': None,
 'endian': 'little',
 'iter': 4632237,
 'runtime': 345540.0,
 'ndim': 3,
 'nparam': 10,
 'nvar': 15,
 'header': 'R R R Mp/cc km/s km/s km/s nT nT nT nPa nT nT nT J/m3 uA/m2 uA/m2 uA/m2_ful33                                                                                                                                                                                                                                                                                                                                                                                                                                       ',
 'g': 1.6666699647903442,
 'c': 5995.7998046875,
 'th': 0.3241580128669739,
 'p1': 3.0,
 'p2': 1.0,
 'p3': 1.0,
 'NX': 6.0,
 'NY': 6.0,
 'NZ': 6.0,
 'R': 2.5,
 'strtime': '0095h59m00.000s'}

In [4]:
mhd.keys()

dict_keys(['grid', 'x', 'y', 'z', 'rho', 'ux', 'uy', 'uz', 'bx', 'by', 'bz', 'p', 'b1x', 'b1y', 'b1z', 'e', 'jx', 'jy', 'jz'])

In [5]:
mhd.attrs

{'file': '../../../data/3d__ful_1_t00955900_n4632237.out',
 'format': None,
 'endian': 'little',
 'iter': 4632237,
 'runtime': 345540.0,
 'ndim': 3,
 'nparam': 10,
 'nvar': 15,
 'header': 'R R R Mp/cc km/s km/s km/s nT nT nT nPa nT nT nT J/m3 uA/m2 uA/m2 uA/m2_ful33                                                                                                                                                                                                                                                                                                                                                                                                                                       ',
 'g': 1.6666699647903442,
 'c': 5995.7998046875,
 'th': 0.3241580128669739,
 'p1': 3.0,
 'p2': 1.0,
 'p3': 1.0,
 'NX': 6.0,
 'NY': 6.0,
 'NZ': 6.0,
 'R': 2.5,
 'strtime': '0095h59m00.000s'}

In [6]:
np.array(mhd['grid'])[0]

755136

In [7]:
mhd.values()

dict_values([dmarray([755136,      1,      1], dtype=int32), dmarray([-224., -216., -224., ...,   59.,   57.,   59.], dtype=float32), dmarray([-44., -44., -44., ...,  45.,  47.,  47.], dtype=float32), dmarray([-44., -44., -36., ...,  47.,  47.,  47.], dtype=float32), dmarray([1.04467, 1.03111, 1.07962, ..., 1.     , 1.     , 1.     ],
        dtype=float32), dmarray([-926.806, -927.173, -926.144, ..., -928.452, -928.45 , -928.45 ],
        dtype=float32), dmarray([266.446, 268.441, 262.65 , ..., 272.956, 272.956, 272.956],
        dtype=float32), dmarray([105.042, 105.898, 100.302, ..., 107.969, 107.969, 107.969],
        dtype=float32), dmarray([ 1.13394e-02,  1.17498e-02, -3.78956e-04, ..., -4.72407e-05,
         -3.12151e-04, -1.45005e-04], dtype=float32), dmarray([2.18344, 2.16405, 2.32134, ..., 1.995  , 1.99497, 1.99499],
        dtype=float32), dmarray([-3.92594, -3.88152, -3.99616, ..., -3.83395, -3.83387, -3.83395],
        dtype=float32), dmarray([0.018886 , 0.0184045, 0.02014

In [8]:
mhdvalues = np.array(mhd['p'])
s = np.array(mhd['grid'])[0]
print(s,mhdvalues)

755136 [0.018886  0.0184045 0.0201401 ... 0.017353  0.0173531 0.0173531]


In [9]:
mhdpoints = np.empty([s, 3], dtype=np.float64)
mhdpoints[:,0] = np.array(mhd['x'])
mhdpoints[:,1] = np.array(mhd['y'])
mhdpoints[:,2] = np.array(mhd['z'])
print(mhdpoints)

[[-224.  -44.  -44.]
 [-216.  -44.  -44.]
 [-224.  -44.  -36.]
 ...
 [  59.   45.   47.]
 [  57.   47.   47.]
 [  59.   47.   47.]]


In [10]:
xxx, yyy, zzz = np.mgrid[-20:20:101j, -20:20:101j, 0:0:1j]

In [11]:
grid1 = griddata(mhdpoints, mhdvalues, (xxx, yyy, zzz), method='nearest')
#grid2 = griddata(mhdpoints, mhdvalues, (xxx, yyy, zzz), method='linear')
#maynotwork#grid3 = griddata(mhdpoints, mhdvalues, (xxx, yyy, zzz), method='cubic')

In [12]:
print(grid1.shape,'\n',xxx.shape)

(101, 101, 1) 
 (101, 101, 1)


In [13]:
x_RE = np.linspace(-20, 20, 81)
y_RE = np.linspace(-20, 20, 81)
z_RE = np.linspace(-20, 20, 81)
qpts = mhdpoints[(mhdpoints[:,0] > (np.amin(x_RE)-1.1)) & 
                 (mhdpoints[:,0] < (np.amax(x_RE)+1.1)) &
                 (mhdpoints[:,1] > (np.amin(y_RE)-1.1)) & 
                 (mhdpoints[:,1] < (np.amax(y_RE)+1.1)) & 
                 (mhdpoints[:,2] > (np.amin(z_RE)-1.1)) & 
                 (mhdpoints[:,2] < (np.amax(z_RE)+1.1))]
qval = mhdvalues[(mhdpoints[:,0] > (np.amin(x_RE)-1.1)) & 
                 (mhdpoints[:,0] < (np.amax(x_RE)+1.1)) &
                 (mhdpoints[:,1] > (np.amin(y_RE)-1.1)) & 
                 (mhdpoints[:,1] < (np.amax(y_RE)+1.1)) & 
                 (mhdpoints[:,2] > (np.amin(z_RE)-1.1)) & 
                 (mhdpoints[:,2] < (np.amax(z_RE)+1.1))]
print(mhdpoints.shape,'\n',qpts.shape,'\n',qval.shape)

(755136, 3) 
 (316008, 3) 
 (316008,)


In [14]:
from kamodo import Kamodo, kamodofy
import time

In [15]:
tic = time.perf_counter()

@kamodofy(units = 'nPa')
def f_XYZ(
    x_RE = np.linspace(-20, 20, 81), 
    y_RE = np.linspace(-20, 20, 81), 
    z_RE = np.linspace(-20, 20, 81)):
    xx, yy, zz = np.meshgrid(x_RE,y_RE,z_RE, indexing = 'xy')

    # Use a mask to reduce the size of the arrays to only values that could impace the new data extraction.
    qpts = mhdpoints[(mhdpoints[:,0] > (np.amin(x_RE)-1.1)) & 
                     (mhdpoints[:,0] < (np.amax(x_RE)+1.1)) &
                     (mhdpoints[:,1] > (np.amin(y_RE)-1.1)) & 
                     (mhdpoints[:,1] < (np.amax(y_RE)+1.1)) & 
                     (mhdpoints[:,2] > (np.amin(z_RE)-1.1)) & 
                     (mhdpoints[:,2] < (np.amax(z_RE)+1.1))]
    qval = mhdvalues[(mhdpoints[:,0] > (np.amin(x_RE)-1.1)) & 
                     (mhdpoints[:,0] < (np.amax(x_RE)+1.1)) &
                     (mhdpoints[:,1] > (np.amin(y_RE)-1.1)) & 
                     (mhdpoints[:,1] < (np.amax(y_RE)+1.1)) & 
                     (mhdpoints[:,2] > (np.amin(z_RE)-1.1)) & 
                     (mhdpoints[:,2] < (np.amax(z_RE)+1.1))]
    result = griddata(qpts, qval, (xx, yy, zz), method='nearest')
    
    # non mask version
    #result = griddata(mhdpoints, mhdvalues, (xx, yy, zz), method='nearest')

    return result

kamodo = Kamodo(f_XYZ = f_XYZ)

toc = time.perf_counter()
print(f"Delta time is {toc - tic:0.4f} seconds")

Delta time is 0.8923 seconds


In [16]:
tic = time.perf_counter()
zval = 0.;
iplot(kamodo.plot(f_XYZ = dict(z_RE = 0)))
toc = time.perf_counter()
print(f"Delta time is {toc - tic:0.4f} seconds")

Delta time is 0.5992 seconds


In [17]:
tic = time.perf_counter()

@kamodofy(units = 'nPa')
def f_XY(
    x_R_E = np.linspace(-40, 20, 300), 
    y_R_E = np.linspace(-20, 20, 200)):
    z_R_E = zval
    xx, yy, zz = np.meshgrid(x_R_E,y_R_E,z_R_E, indexing = 'xy')

    # Use a mask to reduce the size of the arrays to only values that could impace the new data extraction.
    qpts = mhdpoints[(mhdpoints[:,0] > (np.amin(x_R_E)-1.1)) & 
                     (mhdpoints[:,0] < (np.amax(x_R_E)+1.1)) &
                     (mhdpoints[:,1] > (np.amin(y_R_E)-1.1)) & 
                     (mhdpoints[:,1] < (np.amax(y_R_E)+1.1)) & 
                     (mhdpoints[:,2] > (np.amin(z_R_E)-1.1)) & 
                     (mhdpoints[:,2] < (np.amax(z_R_E)+1.1))]
    qval = mhdvalues[(mhdpoints[:,0] > (np.amin(x_R_E)-1.1)) & 
                     (mhdpoints[:,0] < (np.amax(x_R_E)+1.1)) &
                     (mhdpoints[:,1] > (np.amin(y_R_E)-1.1)) & 
                     (mhdpoints[:,1] < (np.amax(y_R_E)+1.1)) & 
                     (mhdpoints[:,2] > (np.amin(z_R_E)-1.1)) & 
                     (mhdpoints[:,2] < (np.amax(z_R_E)+1.1))]
    result = griddata(qpts, qval, (xx, yy, zz), method='linear')
    
    # non mask version
    #result = griddata(mhdpoints, mhdvalues, (xx, yy, zz), method='nearest')
    
    result2=np.reshape(result,(result.shape[0],result.shape[1]))

    return result2

kamodo = Kamodo(f_XY = f_XY)

toc = time.perf_counter()
print(f"Delta time is {toc - tic:0.4f} seconds")

Delta time is 37.9638 seconds


In [18]:
tic = time.perf_counter()
zval = 0.;
iplot(kamodo.plot(f_XY = dict()))
toc = time.perf_counter()
print(f"Delta time is {toc - tic:0.4f} seconds")

Delta time is 39.1536 seconds
